In [1]:
import torch
import math

RoPE 不再为每个位置添加向量，而是：

在向量空间中通过**旋转操作（rotation operation）**，将位置信息“编码进”向量本身。

换句话说，它在计算注意力前，让 query 和 key 旋转不同角度：

$$
Q' = RoPE(Q, pos), \quad K' = RoPE(K, pos)
$$

然后使用下式计算注意力：

$$
Attention = Q'{K'}^{T}
$$

给定一个 token 的向量：

$$
x \in \mathbb{R}^d
$$

RoPE 将它分成两两一组：

$$
x = [x_1, x_2, x_3, x_4, \dots, x_{d-1}, x_d]
$$

每对：

$$
(x_{2i}, \, x_{2i+1})
$$

表示一个二维平面。

对于每个位置 $p$，给定频率向量：

$$
\theta_i = 10000^{-\frac{2i}{d}}
$$

RoPE 定义旋转操作为：

$$
\begin{bmatrix}
x'_{2i} \\
x'_{2i+1}
\end{bmatrix}
=
\begin{bmatrix}
\cos(p \, \theta_i) & -\sin(p \, \theta_i) \\
\sin(p \, \theta_i) & \cos(p \, \theta_i)
\end{bmatrix}
\begin{bmatrix}
x_{2i} \\
x_{2i+1}
\end{bmatrix}
$$

换句话说，就是让每个维度对在平面上**绕原点旋转一个角度**。


### 代码实现

In [8]:
# 生产旋转角度矩阵
def rotary_pos_emb(num_tokens, seq_len):
    dim = num_tokens // 2
    # 频率向量
    inv_freq = 1.0 / (10000 ** (torch.arange(0, num_tokens, 2).float() / num_tokens))
    t = torch.arange(seq_len, dtype = torch.float)
    # 生成频率矩阵
    freqs = torch.einsum("i,j->ij", t, inv_freq)
    cos, sin = freqs.cos(), freqs.sin()
    return cos, sin
def apply_rotary_emb(x, cos, sin):
    x1, x2 = x[..., ::2], x[..., 1::2]
    x_rot = torch.stack((-x2, x1), dim = -1).reshape_as(x)
    return x * cos + x_rot * sin

In [9]:
batch_size, seq_len, d_model = 2, 4, 8
x = torch.randn(batch_size, seq_len, d_model)

cos, sin = rotary_pos_emb(d_model, seq_len)
x_rot = apply_rotary_emb(x, cos[None, :, :], sin[None, :, :])
print(x_rot.shape)

RuntimeError: The size of tensor a (8) must match the size of tensor b (4) at non-singleton dimension 2